## Stationary calibration

This notebook contains the results of calibrating the 4 phones when configured next to each other with the same settings and placed next to each other in the same bookcase.

In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [ ]:
# For plots
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# For maps
import branca.element as bre

In [ ]:
import pandas as pd
import numpy as np
import scipy.interpolate as sci

## Load and validate data

The first issue to note is that we actually have two specs here. The first spec is the checked in `evaluation.spec.sample`, which defines calibration for both stationary and moving instances, and some evaluation trips. However, while starting with the calibration, we noticed some inconsistencies between the power curves. So in order to be more consistent, I defined a second, calibration-only spec `examples/calibration.only.json`, which essentially repeats the calibration experiments multiple times.

After that, I returned to the first set of experiments for the moving calibration and the evaluation.

In [ ]:
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
# If using ServerSpecDetails, data can alternatively be retrieved as such:
# DATASTORE_LOC = "http://localhost:8080"
# sd = eisd.ServerSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, "train_bus_ebike_mtv_ucb")

# You must run `cd bin/ && python dump_data_to_file.py --spec-id sfba_hf_calibration_stationary_only && python dump_data_to_file.py --spec-id sfba_med_freq_calibration_stationary_only`
# before using this notebook!

DATASTORE_LOC = "bin/data/"
sd_hf = eisd.FileSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, "sfba_hf_calibration_stationary_only")
sd_mf = eisd.FileSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, "sfba_med_freq_calibration_stationary_only")

In [ ]:
pv_hf = eipv.PhoneView(sd_hf)

In [ ]:
pv_mf = eipv.PhoneView(sd_mf)

In [ ]:
import importlib
importlib.reload(ezpv)

This validation fails because we forgot to multiply the filter_time by 1000 before setting it.
So we set the value to 1 ms when we meant to set it to 1 sec.
This is not a super bad issue since:
- it only affects android
- due to built-in throttling the data is actually returned only at 1sec frequency or even less anyway
- we don't use this to model anything else, we just use it to help choose regimes for further testing, and for checking that the drain is consistent

We had fixed this bug before the medium frequency collection (phew!) so that does validate properly

In [ ]:
# pv_hf.validate()
pv_mf.validate()

In [ ]:
ev_hf = eiev.EvaluationView()
ev_hf.from_view_multiple_runs(pv_hf, "")

In [ ]:
ev_mf = eiev.EvaluationView()
ev_mf.from_view_multiple_runs(pv_mf, "")

##  Convert the battery drain into a dataframe

This is similar to the eval dataframe, but subtly different

In [ ]:
a2q_map = {"100m": 0, "lowpwr": 0, "10m": 1, "balanced": 1, "best": 2, "high": 2}
f2q_map = {"medium": 1, "high": 2}

In [ ]:
def get_battery_drain_entries(pv):
    battery_entry_list = []
    for phone_os, phone_map in pv.map().items():
        print(15 * "=*")
        print(phone_os, phone_map.keys())
        for phone_label, phone_detail_map in phone_map.items():
            print(4 * ' ', 15 * "-*")
            print(4 * ' ', phone_label, phone_detail_map.keys())
            # this spec does not have any calibration ranges, but evaluation ranges are actually cooler
            for r in phone_detail_map["calibration_ranges"]:
                print(8 * ' ', 30 * "=")
                print(8 * ' ',r.keys())
                print(8 * ' ',r["trip_id"], r["trip_run"])
                bcs = r["battery_df"]["battery_level_pct"]
                delta_battery = bcs.iloc[0] - bcs.iloc[-1]
                if "medium_freq" or "high_frequency" in r["trip_id"]:
                    cc = r["trip_id"].split("_")
                    config_map = {"accuracy_str": cc[0] if phone_os == "ios" else cc[1],
                                  "frequency_str": cc[3],
                    }
                    config_map["accuracy"] = a2q_map[config_map["accuracy_str"]]
                    config_map["frequency"] = f2q_map[config_map["frequency_str"]]
                print("Battery starts at %d, ends at %d, drain = %d" % (bcs.iloc[0], bcs.iloc[-1], delta_battery))
                battery_entry = {"phone_os": phone_os, "phone_label": phone_label, "config": r["trip_id"],
                                 "run": r["trip_run"], "duration": r["duration"],
                                 "battery_drain_observed": delta_battery}
                battery_entry.update(config_map)
                battery_entry_list.append(battery_entry)
    return battery_entry_list

In [ ]:
battery_entries_list = []
battery_entries_list.extend(get_battery_drain_entries(pv_hf))
battery_entries_list.extend(get_battery_drain_entries(pv_mf))
battery_drain_df = pd.DataFrame(battery_entries_list)

In [ ]:
battery_drain_df.head()

## Normalize

It turns out that the calibration ranges have had very different durations. So if we want to really standardize them, we need to calculate the drain for the same duration. But what should that duration be? (min, max, mean)? For now, we pick the mean duration and because the points are coarse, use curve fitting to ensure that we have a point at that mean duration

In [ ]:
print("mean = %s hours, min = %s" % (battery_drain_df.duration.mean() / (60*60), battery_drain_df.duration.min() / (60*60)))
battery_drain_df.duration.describe()

In [ ]:
def explore_interpolated_battery_drain_at_duration(pv, hr_from_start, i = 0):
    bad_extrapolations = []
    for phone_os, phone_map in pv.map().items():
        print(15 * "=*")
        print(phone_os, phone_map.keys())
        for phone_label, phone_detail_map in phone_map.items():
            print(4 * ' ', 15 * "-*")
            print(4 * ' ', phone_label, phone_detail_map.keys())
            for r in phone_detail_map["calibration_ranges"]:
                print(8 * ' ', 30 * "=")
                print(8 * ' ',r.keys())
                print(8 * ' ',i, r["trip_id"], r["trip_run"])
                battery_df = r["battery_df"]
                battery_fn = sci.interp1d(battery_df.hr, battery_df.battery_level_pct, fill_value="extrapolate")
                hr_range = pd.Series(np.arange(battery_df.hr.min(), hr_from_start, 60 / (60 * 60))) # 1 minute
                
                interp_battery_level = pd.Series(battery_fn(hr_range))
                changes = interp_battery_level.diff()
                extrapolated_changes = changes[hr_range > battery_df.hr.max()]
                print(len(extrapolated_changes), np.count_nonzero(extrapolated_changes))
                print("Extrapolating from %s -> %s, added %d more points" % (battery_df.hr.max(), hr_from_start, len(extrapolated_changes)))
                if np.count_nonzero(extrapolated_changes) < 0.1 * len(extrapolated_changes):
                    print("WARNING: BAD EXTRAPOLATION")
                    bad_extrapolations.append(phone_label+"_"+r["trip_id"])
                axes_flat[i].plot(battery_df.hr, battery_df.battery_level_pct, color="green")
                axes_flat[i].plot(hr_range, interp_battery_level, color="red")
                axes_flat[i].set_title(phone_label+"_"+r["trip_id"])
                i = i+1
                
    return (i, bad_extrapolations)

### Trying to use the mean of the durations, which will require some extrapolation

In [ ]:
row_count = int(len(battery_drain_df) / 3) + 1
ifig, axes_array = plt.subplots(nrows=row_count, ncols=3, figsize=(20, 120), sharex=True, sharey=True)
axes_flat = axes_array.flatten()
all_be = []
(i, be) = explore_interpolated_battery_drain_at_duration(pv_hf, 12)
all_be.extend(be)
(i, be) = explore_interpolated_battery_drain_at_duration(pv_mf, 12, i)
all_be.extend(be)
all_be

In [ ]:
len(all_be)

We can see from the graphs here that some extrapolations just don't work well. I am not sure if this is because there are too few points or something else weird in the spline code. For example, `100m_lowpwr_accuracy_high_frequency_stationary_0` bottoms out at `93%`

#### Detailed exploration

In [ ]:
battery_df = [r for r in pv_hf.map()["android"]["ucb-sdb-android-2"]["calibration_ranges"] if r["trip_id"] == "100m_lowpwr_accuracy_high_frequency_stationary_0"][0]["battery_df"]

In [ ]:
battery_df.hr.max()

In [ ]:
                battery_fn = sci.interp1d(battery_df.hr, battery_df.battery_level_pct, fill_value="extrapolate")
                hr_range = pd.Series(np.arange(battery_df.hr.min(), 12, 60 / (60 * 60))) # 1 minute
                interp_battery_level = pd.Series(battery_fn(hr_range))
                changes = interp_battery_level.diff(); changes.tail()
                extrapolated_changes = changes[hr_range > battery_df.hr.max()]; extrapolated_changes.tail()
                print("Extrapolating from %s -> %s, added %d more points" % (battery_df.hr.max(), 12, len(extrapolated_changes)))
                print(len(extrapolated_changes), np.count_nonzero(extrapolated_changes))

By adding a simple similar check, we can see that a 45/116 curves have bad extrapolations. So we can't really use extrapolation in this case

### Switching to the min of the durations, which will give us less data

In [ ]:
row_count = int(len(battery_drain_df) / 3) + 1
ifig, axes_array = plt.subplots(nrows=row_count, ncols=3, figsize=(20, 120), sharex=True, sharey=True)
axes_flat = axes_array.flatten()
all_be = []
(i, be) = explore_interpolated_battery_drain_at_duration(pv_hf, 5.9993)
all_be.extend(be)
(i, be) = explore_interpolated_battery_drain_at_duration(pv_mf, 5.9993, i)
all_be.extend(be)
print(len(all_be))
all_be

In [ ]:
battery_df = [r for r in pv_hf.map()["android"]["ucb-sdb-android-2"]["calibration_ranges"] if r["trip_id"] == "best_high_accuracy_high_frequency_stationary_0"][0]["battery_df"]
battery_fn = sci.interp1d(battery_df.hr, battery_df.battery_level_pct, fill_value="extrapolate")
hr_range = pd.Series(np.arange(battery_df.hr.min(), 12, 60 / (60 * 60))) # 1 minute
interp_battery_level = pd.Series(battery_fn(hr_range))
changes = interp_battery_level.diff(); changes.tail()
extrapolated_changes = changes[hr_range > battery_df.hr.max()]; extrapolated_changes.tail()
print("Extrapolating from %s -> %s, added %d more points" % (battery_df.hr.max(), 12, len(extrapolated_changes)))
print(len(extrapolated_changes), np.count_nonzero(extrapolated_changes))

In [ ]:
ifig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16,8))
ax.plot(battery_df.hr, battery_df.battery_level_pct, color="green")
ax.plot(hr_range, interp_battery_level, color="red")

## Now to actually interpolate based on the min

In [ ]:
def get_interpolated_battery_drain_at_duration(pv, hr_from_start):
    interpolated_battery_levels = []
    for phone_os, phone_map in pv.map().items():
        print(15 * "=*")
        print(phone_os, phone_map.keys())
        for phone_label, phone_detail_map in phone_map.items():
            print(4 * ' ', 15 * "-*")
            print(4 * ' ', phone_label, phone_detail_map.keys())
            for r in phone_detail_map["calibration_ranges"]:
                print(8 * ' ', 30 * "=")
                print(8 * ' ',r.keys())
                print(8 * ' ',r["trip_id"], r["trip_run"])
                battery_df = r["battery_df"]
                battery_fn = sci.interp1d(battery_df.hr, battery_df.battery_level_pct, fill_value="extrapolate")
                level_at_hr_from_start = battery_fn(hr_from_start)
                print("observed delta = %s at %s, predicted delta = %s at %s" % 
                      (battery_df.battery_level_pct.iloc[-1], battery_df.hr.max(),
                       level_at_hr_from_start, hr_from_start))
                interpolated_battery_levels.append((battery_df.battery_level_pct.iloc[0] - level_at_hr_from_start))
    return interpolated_battery_levels

In [ ]:
all_interpolated_battery_levels = []
all_interpolated_battery_levels.extend(get_interpolated_battery_drain_at_duration(pv_hf, battery_drain_df.duration.min()/(60 * 60)))
all_interpolated_battery_levels.extend(get_interpolated_battery_drain_at_duration(pv_mf, battery_drain_df.duration.min()/(60 * 60)))
battery_drain_df["battery_drain"] = all_interpolated_battery_levels

In [ ]:
battery_drain_df["phone_idx"] = battery_drain_df.phone_label.apply(lambda l: l.split("-")[-1])

In [ ]:
battery_drain_df.head()

In [ ]:
battery_drain_df.replace("balanced", "balpwr", inplace=True)
battery_drain_df.replace("medium", "med", inplace=True)

In [ ]:
ifig, ax_array = plt.subplots(nrows=1,ncols=2,figsize=(12,6), sharex=False, sharey=True)
battery_drain_df.query("phone_os == 'android'").boxplot(ax=ax_array[0], column=["battery_drain"], by=["accuracy_str", "frequency_str"])
battery_drain_df.query("phone_os == 'ios'").boxplot(ax=ax_array[1], column=["battery_drain"], by=["accuracy_str", "frequency_str"])
for ax in ax_array:
    ax.set_xlabel("[accuracy, frequency]")

ax_array[0].set_title("android")
ax_array[1].set_title("ios")
ax_array[0].set_ylabel("Battery drain (%)")
ifig.suptitle("Boxplot of battery drain for various combinations of accuracy and frequency")

# ifig.tight_layout()

In [ ]:
ifig, ax_array = plt.subplots(nrows=2,ncols=1,figsize=(16,8), sharex=False, sharey=True, )
battery_drain_df.query("phone_os == 'android'").boxplot(ax=ax_array[0], column=["battery_drain"], by=["accuracy", "frequency", "phone_idx"])
battery_drain_df.query("phone_os == 'ios'").boxplot(ax=ax_array[1], column=["battery_drain"], by=["accuracy", "frequency", "phone_idx"])

for ax in ax_array:
    ax.set_title("")
    ax.set_ylabel("Battery drain")

ax_array[0].set_title("accuracy = {0: lowpwr, 1: balanced, 2: high}, frequency = {1: medium, 2: high}")
ax_array[1].set_title("accuracy = {0: 100m, 1: 10m, 2: high}, frequency = {1: medium, 2: high}")
ifig.tight_layout(pad=3)

In [ ]:
battery_drain_df.query("phone_os == 'ios' & accuracy == 1 & frequency == 2 & phone_label == 'ucb-sdb-ios-4'").battery_drain.describe()

In [ ]:
ifig, ax_array = plt.subplots(nrows=2,ncols=1,figsize=(16,8), sharex=False, sharey=True)
battery_drain_df.query("phone_os == 'android'").boxplot(ax=ax_array[0], column=["battery_drain"], by=["accuracy", "frequency", "phone_idx"])
battery_drain_df.query("phone_os == 'ios'").boxplot(ax=ax_array[1], column=["battery_drain"], by=["accuracy", "frequency", "phone_idx"])

for ax in ax_array:
    ax.set_title("")
    ax.set_ylabel("Battery drain")

ax_array[0].set_title("accuracy = {0: lowpwr, 1: balanced, 2: high}, frequency = {1: medium, 2: high}")
ax_array[1].set_title("accuracy = {0: 100m, 1: 10m, 2: high}, frequency = {1: medium, 2: high}")

In [ ]:
ifig, ax_array = plt.subplots(nrows=2,ncols=2,figsize=(16,8), sharex=False, sharey=True)
battery_drain_df.query("phone_os == 'android' & accuracy==2 & frequency==2").boxplot(ax=ax_array[0][0], column=["battery_drain"], by=["run"])
battery_drain_df.query("phone_os == 'ios' & accuracy==2 & frequency==2").boxplot(ax=ax_array[1][0], column=["battery_drain"], by=["run"])
battery_drain_df.query("phone_os == 'android' & accuracy==2 & frequency==2").boxplot(ax=ax_array[0][1], column=["battery_drain"], by=["phone_idx"])
battery_drain_df.query("phone_os == 'ios' & accuracy==2 & frequency==2").boxplot(ax=ax_array[1][1], column=["battery_drain"], by=["phone_idx"])

# for ax in ax_array:
#     ax.set_title("")
#     ax.set_ylabel("Battery drain")

ifig.tight_layout(pad=2)